In [1]:
import livef1
from livef1.utils.constants import interpolation_map
from livef1.adapters.livetimingf1_adapter import livetimingF1_getdata

import pandas as pd
import numpy as np
from datetime import timedelta

import matplotlib.pyplot as plt

session = livef1.get_session(season=2024, meeting_identifier="Spa", session_identifier="Race")

2025-01-15 21:57:37,190 : livef1 : INFO :: Selected meeting/session is:
Meeting Offname : FORMULA 1 ROLEX BELGIAN GRAND PRIX 2024
Meeting Name : Belgian Grand Prix
Meeting Circuit Shortname : Spa-Francorchamps
2025-01-15 21:57:37,191 : livef1 : INFO :: The meeting was received successfully.
2025-01-15 21:57:37,194 : livef1 : INFO :: Selected meeting/session is:
Session Name : Race
2025-01-15 21:57:37,194 : livef1 : INFO :: The session was received successfully.


In [2]:
res = session.get_data(dataName="Timing_Data")
df_exp = pd.DataFrame(res.value)
df_exp

2025-01-15 21:57:38,132 : livef1 : INFO :: Getting requested data.
Selected session : 2024 Belgian Grand Prix Race
Topic : TimingData
2025-01-15 21:57:39,087 : livef1 : INFO :: Data is successfully received.
2025-01-15 21:57:39,224 : livef1 : INFO :: Data is successfully parsed.


,SessionKey,timestamp,DriverNo,GapToLeader,IntervalToPositionAhead_Value,IntervalToPositionAhead_Catching,Line,Position,ShowPosition,RacingNumber,...,Sectors_2_Segments_6_Status,Sectors_1_PreviousValue,NumberOfLaps,Sectors_2_PreviousValue,Sectors_0_Value,Sectors_0_OverallFastest,Sectors_0_PersonalFastest,Sectors_0_PreviousValue,BestLapTime_Lap,NumberOfPitStops
0,9574,00:00:03.734,16,,,False,1.0,1,True,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9574,00:00:03.734,11,,,False,2.0,2,True,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9574,00:00:03.734,44,,,False,3.0,3,True,44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9574,00:00:03.734,4,,,False,4.0,4,True,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9574,00:00:03.734,81,,,False,5.0,5,True,81,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53035,9574,02:17:07.716,1,NaN,NaN,True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53036,9574,02:17:07.716,10,NaN,NaN,True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53037,9574,02:17:07.716,77,NaN,NaN,True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53038,9574,02:17:07.716,20,NaN,NaN,True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df_11 = df_exp[df_exp["DriverNo"] == "11"]
# df_11 = df_exp[df_exp["DriverNo"] == "24"]

In [11]:
sector_cols = {
    "Sectors_0_Value" : "sector1_time",
    "Sectors_1_Value" : "sector2_time",
    "Sectors_2_Value" : "sector3_time",
    "Sectors_0_PreviousValue" : None,
    "Sectors_1_PreviousValue" : None,
    "Sectors_2_PreviousValue" : None
}

segment_cols = {col:f"S{col.split('_')[1]}_segment{col.split('_')[3]}_status" for col in df_11.columns if ("Sectors" in col) & ("Segments" in col)}

speedTrap_cols = {
    "Speeds_I1_Value" : "speed_I1",
    "Speeds_I2_Value" : "speed_I2",
    "Speeds_FL_Value" : "speed_FL",
    "Speeds_ST_Value" : "speed_ST",
}
pit_cols = {
    "InPit" : "in_pit",
    "PitOut" : "pit_out"
}

base_cols = {
    "NumberOfLaps" : "lap_number",
    "LastLapTime_Value" : "lap_time"
    }

extra_cols = ["no_pits"]

col_map = {**base_cols, **pit_cols, **sector_cols, **speedTrap_cols} #, **segment_cols}
cols = list(base_cols.values()) + list(pit_cols.values()) + list(sector_cols.values()) + list(speedTrap_cols.values())  # + list(segment_cols.values())
raw_cols = list(base_cols.keys()) + list(pit_cols.keys()) + list(sector_cols.keys()) + list(speedTrap_cols.keys()) # + list(segment_cols.keys())


def str_timedelta(x):
    if isinstance(x,str):
        count_sep = x.count(":")

        if count_sep == 0:
            return "00:00:" + x
        elif count_sep == 1:
            return "00:" + x
        else: return x
    else:
        return x


df_test = df_11[["timestamp"] + raw_cols].dropna(subset=raw_cols, how="all").replace('',np.nan)


for col in ["Sectors_0_Value","Sectors_1_Value","Sectors_2_Value", "Sectors_0_PreviousValue","Sectors_1_PreviousValue","Sectors_2_PreviousValue", "LastLapTime_Value"]:
    df_test[col] = df_test[col]
    df_test[col] = pd.to_timedelta(df_test[col].apply(str_timedelta))

def enter_new_lap(laps, record):
    print(">>", record)
    if laps is None and record is None:
        no_pits = 0
        laps = []
        record = {key:None if key != "lap_number" else 1 for key in cols}
        record["no_pits"] = no_pits
        return [], record, timedelta(seconds=0)

    if record["lap_time"] is None:
        record["lap_time"] = record["sector1_time"] + record["sector2_time"] + record["sector3_time"]

    laps.append(record)

    no_pits = record["no_pits"]
    record = {key:None if key != "lap_number" else val+1 for key, val in record.items()}
    record["no_pits"] = no_pits

    return laps, record


new_lap_allowed = True
laps, record, last_record_ts = enter_new_lap(None, None)

for idx,row in df_test.iterrows():
    ts = pd.to_timedelta(row.timestamp)
    
    for sc_key, sc_value in row[list(speedTrap_cols.keys()) + ["LastLapTime_Value"]].dropna().to_dict().items():
        record[col_map[sc_key]] = sc_value
    
    for sc_key, sc_value in row[list(pit_cols.keys())].dropna().to_dict().items():
        if sc_key == "InPit":
            if sc_value == 1:
                record[col_map[sc_key]] = ts
            # else:
            #     record[col_map[sc_key]] = None
        elif sc_key == "PitOut":
            if sc_value == True:
                record[col_map[sc_key]] = ts
                record["no_pits"] += 1
    
    for sc_key, sc_value in row[list(sector_cols.keys())].dropna().to_dict().items():
        sc_no = int(sc_key.split("_")[1])
        key_type = sc_key.split("_")[2]
        # print(sc_no, "||", ts, "||", key_type, sc_value)

        if key_type == "Value":
            if record[f"sector{str(sc_no + 1)}_time"] == None:
                record[f"sector{str(sc_no + 1)}_time"] = sc_value
                last_record_ts = ts
                if sc_no == 2:
                    # entering new lap
                    laps, record = enter_new_lap(laps, record)
            elif sc_value == record[str(sc_no + 1)]: pass
            elif ts - last_record_ts > timedelta(seconds=10):
                # entering new lap
                laps, record = enter_new_lap(laps, record)
                record[f"sector{str(sc_no + 1)}_time"] = sc_value
                last_record_ts = ts

        elif key_type == "PreviousValue" and ts - last_record_ts > timedelta(seconds=10):
            record[f"sector{str(sc_no + 1)}_time"] = sc_value
            last_record_ts = ts
            if sc_no == 2:
                # entering new lap
                laps, record = enter_new_lap(laps, record)

laps_df = pd.DataFrame(laps)

segments = ["sector1_time","sector2_time","sector3_time"]
for idx in range(len(segments)):
    rest = np.delete(segments, idx)
    laps_df[segments[idx]] = (laps_df[segments[idx]].fillna(timedelta(minutes=0)) + (laps_df[segments[idx]].isnull() & (laps_df["lap_number"] > 1)) * (laps_df[segments[idx]].isnull() * (laps_df["lap_time"].fillna(timedelta(minutes=0)) - laps_df[rest].sum(axis=1)))).replace(timedelta(minutes=0), np.timedelta64("NaT"))

laps_df

>> None
>> {'lap_number': 1, 'lap_time': None, 'in_pit': Timedelta('0 days 00:18:02.529000'), 'pit_out': None, 'sector1_time': None, 'sector2_time': Timedelta('0 days 00:00:50.038000'), 'sector3_time': Timedelta('0 days 00:00:29.491000'), None: None, 'speed_I1': '316', 'speed_I2': '201', 'speed_FL': '216', 'speed_ST': '310', 'no_pits': 0}
>> {'lap_number': 2, 'lap_time': Timedelta('0 days 00:01:50.240000'), 'in_pit': None, 'pit_out': None, 'sector1_time': Timedelta('0 days 00:00:31.365000'), 'sector2_time': Timedelta('0 days 00:00:49.207000'), 'sector3_time': Timedelta('0 days 00:00:29.668000'), None: None, 'speed_I1': '325', 'speed_I2': '202', 'speed_FL': '217', 'speed_ST': '308', 'no_pits': 0}
>> {'lap_number': 3, 'lap_time': Timedelta('0 days 00:01:49.983000'), 'in_pit': None, 'pit_out': None, 'sector1_time': Timedelta('0 days 00:00:31.470000'), 'sector2_time': Timedelta('0 days 00:00:48.913000'), 'sector3_time': Timedelta('0 days 00:00:29.600000'), None: None, 'speed_I1': '314', 's

,lap_number,lap_time,in_pit,pit_out,sector1_time,sector2_time,sector3_time,None,speed_I1,speed_I2,speed_FL,speed_ST,no_pits
0,1,NaT,0 days 00:18:02.529000,NaT,NaT,0 days 00:00:50.038000,0 days 00:00:29.491000,None,316,201,216,310,0
1,2,0 days 00:01:50.240000,NaT,NaT,0 days 00:00:31.365000,0 days 00:00:49.207000,0 days 00:00:29.668000,None,325,202,217,308,0
2,3,0 days 00:01:49.983000,NaT,NaT,0 days 00:00:31.470000,0 days 00:00:48.913000,0 days 00:00:29.600000,None,314,201,218,309,0
3,4,0 days 00:01:49.790000,NaT,NaT,0 days 00:00:31.356000,0 days 00:00:49.032000,0 days 00:00:29.402000,None,320,199,218,None,0
4,5,0 days 00:01:49.896000,NaT,NaT,0 days 00:00:31.447000,0 days 00:00:48.958000,0 days 00:00:29.491000,None,319,197,215,311,0
5,6,0 days 00:01:50.129000,NaT,NaT,0 days 00:00:31.583000,0 days 00:00:49.116000,0 days 00:00:29.430000,None,312,198,215,None,0
6,7,0 days 00:01:49.637000,NaT,NaT,0 days 00:00:31.524000,0 days 00:00:48.599000,0 days 00:00:29.514000,None,321,200,215,309,0
7,8,0 days 00:01:49.912000,NaT,NaT,0 days 00:00:31.530000,0 days 00:00:48.828000,0 days 00:00:29.554000,None,314,199,216,310,0
8,9,0 days 00:01:50.286000,NaT,NaT,0 days 00:00:31.506000,0 days 00:00:49.173000,0 days 00:00:29.607000,None,315,199,216,None,0
9,10,0 days 00:01:50.057000,NaT,NaT,0 days 00:00:31.598000,0 days 00:00:48.943000,0 days 00:00:29.516000,None,314,198,216,309,0


In [ ]:
# 2048 - Yellow


In [3]:
url = session.path + "TimingData.jsonStream"
res = livetimingF1_getdata(url, True)

In [4]:
for rec in res:
    if rec[0] == '02:12:25.875':
        print(rec[1])

{'Lines': {'11': {'InPit': False, 'PitOut': True, 'Status': 608, 'Sectors': {'0': {'Value': ''}, '1': {'Value': ''}, '2': {'Value': ''}}, 'Speeds': {'I1': {'Value': ''}, 'I2': {'Value': ''}, 'ST': {'Value': ''}}}, '22': {'Sectors': {'1': {'PreviousValue': '48.412'}}}}}
{'Lines': {'14': {'Sectors': {'2': {'Segments': {'6': {'Status': 2048}}}}}}}
{'Lines': {'77': {'GapToLeader': '+69.509', 'IntervalToPositionAhead': {'Value': '+4.663'}}}}
